# Perception Node Tasarımı (LibTorch + OpenCV) – Teorik İskelet

Bu not defteri, **LibTorch + OpenCV** tabanlı tüm algılama (perception) kodunu
*fonksiyon yığını* olmaktan çıkarıp **gerçek bir modüle (Perception Node)** dönüştürmek için teorik iskeleti anlatır.

Odak noktamız:

- `ImageFrame` veri tipi
- `DetectedObject` veri tipi
- `PerceptionNode` sınıfı
- `constructor`, `preprocess()`, `inference()`, `postprocess()`, `process()` akışı
- CNN / YOLO gibi modellerin bu iskelete nasıl oturduğu
- Gerçek zamanlı sistemlerde neden bu yapının tercih edildiği


## 1. Perception Node Nedir? Sistem İçindeki Yeri

Otonom sürüş / robotik mimaride tipik akış:

1. **Sensor** (kamera, LiDAR, radar…)
2. **Perception** (algılama)
3. **Prediction / Planning** (tahmin & planlama)
4. **Control** (direksiyon, fren, gaz)

Bizim modülümüz: **Perception Node**.

- **Girdi (input):** Kamera görüntüsü (ve ilgili metadata)
- **Çıktı (output):** Tespit edilen nesneler (sınıf + konum + skor)
- **İdeal API:**

  ```cpp
  auto detections = perception.process(frame);
  ```

Dışarıdan bakan için önemli olan tek şey:

- Bir `ImageFrame` veriyorum.
- Bana bir `DetectedObject` listesi dönüyor.

İçeride hangi modelin kullanıldığı, kaç adım preprocess olduğu, NMS nasıl yapıldığı dışarıdan **görünmez**.
Bu soyutlama, büyük sistemlerde tasarımın temiz kalmasını sağlar.


## 2. Neden Sınıf / Modül? Neden Fonksiyon Yığını Değil?

Klasik kötü senaryo:

- `main.cpp` içinde onlarca fonksiyon:
  - `load_model(...)`
  - `preprocess(...)`
  - `run_inference(...)`
  - `postprocess(...)`
  - `draw_boxes(...)`
  - büyük bir `while (true)` döngüsü

Bu yaklaşım şu problemleri doğurur:

1. **Bakım zorluğu**
   - Tüm mantık tek dosyada toplanır.
   - Kod büyüdükçe okunabilirlik düşer.

2. **Tekrar kullanım yok**
   - Aynı pipeline’ı başka projede kullanmak istersen,
     `main.cpp` kopyala–yapıştır yapman gerekir.

3. **Gerçek zamanlı mimarilere taşımak zor**
   - Multi-thread, ROS2 node, pipeline yapıları kurarken,
     dağınık fonksiyonlarla uğraşmak zorlaşır.
   - Örneğin ROS2 callback içinde sadece `perception.process(frame)` demek istersin.

4. **Kaynak yönetimi dağınık**
   - Model açıkça nereye ait belli değildir.
   - `torch::jit::script::Module` yükleme, device seçimi, buffer yönetimi gibi işler,
     kodun her yerine dağılır.

Bu nedenle algılama mantığını tek bir sınıfta toplarız:

- Model yükleme → **constructor**
- Girdi–çıktı akışı → `process()`
- İç detaylar → private fonksiyonlar (`preprocess`, `inference`, `postprocess`)

Böylece:

- Dışarıdan basit bir arayüz görürüz.
- İçerisi modüler ve test edilebilir kalır.


## 3. ImageFrame Veri Tipi

Gerçek bir sistemde tek bilgi sadece görüntü değildir.
Her frame ile birlikte şu bilgiler de önemli olur:

- `image`: `cv::Mat` ham görüntü
- `timestamp`: bu frame ne zaman çekildi?
- `frame_id`: frame numarası veya benzersiz kimlik
- `camera_id` / `camera_name`: hangi kameradan geldi?
- Gerekirse kamera parametreleri (kalibrasyon bilgileri vb.)

**Neden sadece `cv::Mat` kullanmıyoruz?**

- **Senkronizasyon:** Birden fazla sensör olduğunda zaman bilgisi önemlidir.
- **Loglama ve debugging:** Hangi frame’de hangi hatalar oldu kolayca takip edilir.
- **Latency ölçümleri:** `now - frame.timestamp` ile algılama gecikmesi hesaplanabilir.
- **ROS2 / mesajlaşma sistemlerinde header ihtiyacı:** timestamp + frame id bilgisi gerekir.

Özet:

> **ImageFrame = Görüntü + bağlam (context).**

Bu yapı sayesinde tek bir parametre ile hem resmi hem de ilgili tüm metadata’yı taşıyabiliriz.


## 4. DetectedObject Veri Tipi

Bir nesne tespit edildiğinde tipik olarak şu bilgiler tutulur:

- `class_id` → modelin sayısal sınıf kimliği
- `class_name` → insan okunabilir sınıf ismi (person, car, truck…)
- `score` → güven skoru (0.0–1.0 arası)
- `bbox` → sınır kutusu (x, y, width, height) piksel cinsinden

Opsiyonel alanlar:

- `track_id` → zaman içinde aynı nesneyi takip etmek için ID
- `distance` / 3D konum bilgisi
- `source_frame_id` → hangi frame’den geldiği

**Neden ayrı bir tip olarak tanımlıyoruz?**

- Diğer modüller (planning, UI, logging) tensor ile değil,
  **algılanmış nesne** kavramı ile çalışmak ister.
- Model değişse bile (CNN, YOLO, transformer), dışarıya
  aynı `DetectedObject` tipini döndürebilmek, kodun geri kalanını
  modelden bağımsız hale getirir.

Özet:

> **DetectedObject = Model çıktısının anlamlı, yüksek seviyeli temsili.**


## 5. PerceptionNode Sınıfı – Genel Sorumluluk

`PerceptionNode`, tüm algılama pipeline’ını tek bir modülde toplar.

Ana görevleri:

- TorchScript modeli yüklemek ve yaşam döngüsünü yönetmek.
- Girdi olarak `ImageFrame` alıp, çıktıda `DetectedObject` listesi döndürmek.
- Preprocess, inference, postprocess adımlarını **içeride** organize etmek.
- Dışarıya sadece **tek bir fonksiyon** sunmak:

  ```cpp
  std::vector<DetectedObject> process(const ImageFrame& frame);
  ```

Bu sayede:

- Konfigürasyon, model yükleme, cihaz seçimi gibi detaylar gizlenir.
- Gerçek zamanlı döngü, ROS2 node veya başka bir pipeline,
  sadece bu fonksiyona odaklanır.


### 5.1. PerceptionNode İçinde Tutulacak Üyeler

Tipik olarak `PerceptionNode` içinde şu üyeler bulunur:

- `torch::jit::script::Module model`
  - TorchScript modelin kendisi.
  - Constructor’da yüklenir, program süresince hayatta kalır.

- `torch::Device device`
  - CPU veya CUDA seçimi.
  - Tüm tensor oluşturma ve `to()` çağrıları bu cihaza göre yapılır.

- Model ile ilgili parametreler:
  - Giriş görüntü boyutu (örn. 640x640)
  - Normalize için mean / std değerleri
  - YOLO ise IoU threshold, skor threshold
  - Sınıf isim listesi (`std::vector<std::string> class_names`)

- Performans için yardımcı yapılar (opsiyonel):
  - Önceden allocate edilmiş input tensor
  - Geçici `std::vector<DetectedObject>` buffer’ı

Amaç:

- Model sadece **bir kez** yüklenir.
- Her `process()` çağrısında minimum ek yük (allocation) yaratılır.
- Gerçek zamanlı sistemlerde daha stabil ve öngörülebilir süreler elde edilir.


### 5.2. Constructor – Model Yükleme ve Başlangıç

Constructor tipik olarak şu işleri yapar:

- TorchScript model dosyasının path’ini alır.
- `torch::jit::load(...)` ile modeli yükler.
- `device` seçer (CPU / CUDA).
- `model.to(device)` ile modeli doğru cihaza taşır.
- `model.eval()` ile modeli inference moduna alır.
- İsteğe bağlı olarak 1–2 kez dummy input ile **warm-up** yapar.

Neden önemlidir?

- Model yükleme pahalı bir işlemdir, sadece başlangıçta yapılmalıdır.
- Warm-up sayesinde ilk gerçek inference çağrısında gecikme azalır.
- Gerçek zamanlı döngü, sürekli model yükleme maliyetine maruz kalmaz.

Özet:

> Constructor, `PerceptionNode` kurulduğunda modülü
> **hazır ve sıcak** hale getirir. `process()` buna dayanır.


### 5.3. preprocess() – cv::Mat → torch::Tensor

Görev:

- `ImageFrame` içindeki `cv::Mat` görüntüyü alır.
- Modelin beklediği formata dönüştürür:

Tipik adımlar:

- Boyutlandırma (resize veya letterbox – YOLO için 640x640 gibi)
- Renk uzayı dönüşümü (BGR → RGB)
- Tip dönüşümü (`uint8` → `float32`)
- Normalizasyon ([0,255] → [0,1], ardından mean/std)
- Veri düzeni (H x W x C → C x H x W)
- Batch dimension ekleme (1 x C x H x W)

Neden ayrı bir fonksiyon?

- Preprocess mantığı görüntüye özeldir ve modelden büyük ölçüde bağımsızdır.
- Test etmek, profil çıkarmak, ileride GPU hızlandırma eklemek kolaylaşır.
- CNN, YOLO, transformer gibi farklı modellere geçerken çoğu zaman
  preprocess yapısı benzerdir, küçük farklılıklar izole edilebilir.

Özet:

> **preprocess():** OpenCV dünyasından **PyTorch tensor** dünyasına geçiş kapısı.


### 5.4. inference() – Girdi Tensor → Ham Model Çıktısı

Görev:

- Preprocess çıktısı olan input tensor’ü alır.
- `model.forward()` ile inference yapar.
- Modelin ham çıktısını tensor veya tensor listesi olarak döndürür.

Önemli noktalar:

- `torch::NoGradGuard` kullanarak gradient hesaplamalarını kapatmak
  (inference performansı için).
- Model türüne göre çıktı formatını bilmek gerekir:
  - CNN classifier: `output.shape = [batch, num_classes]`
  - YOLO benzeri detector: `output.shape = [N, 85]` gibi (bbox + skorlar)

Neden ayrı bir fonksiyon?

- Bu katman en ağır hesap yükünün olduğu yerdir.
- Profiling, latency ölçümleri, device geçişleri bu bölümde izlenir.
- Alt tarafta CNN → YOLO değişse bile inference çağrı yapısı benzer kalır,
  asıl format farkları `postprocess()` içinde çözülür.


### 5.5. postprocess() – Ham Çıkış → DetectedObject Listesi

Görev:

- Modelden gelen ham tensorleri, `std::vector<DetectedObject>` içine
  dönüştürmek.

İki örnek üzerinden düşünebiliriz:

#### CNN Classifier

- Çıktı: `[1, num_classes]` şeklinde bir vektör.
- İşlem:
  - `argmax` ile en yüksek skorlu sınıfı bul.
  - Skoru ve sınıf id’sini al.
  - Tüm resmi kapsayan veya mantıklı bir bbox ile tek bir `DetectedObject` oluştur.

#### YOLO / Nesne Tespit Modeli

- Çıktı: `[N, 85]` gibi (x, y, w, h, obj_score, class_scores...)
- Tipik adımlar:
  - Bbox koordinatlarını çöz (merkez + genişlik/yükseklik → köşe koordinatları).
  - Skor eşiği uygula (score threshold).
  - Non-Maximum Suppression (NMS) uygula.
  - Kalan kutular için `DetectedObject` oluştur:
    - `bbox`
    - `class_id`
    - `class_name`
    - `score`

Neden ayrı bir fonksiyon?

- Postprocess tamamen **modelin çıktı formatına** bağlıdır.
- Aynı `preprocess + inference` iskeletiyle, farklı modeller için
  farklı `postprocess()` implementasyonları yazılabilir.
- CPU yoğunluğu yüksek olan NMS, filtreleme gibi işlemlerin kontrolü
  bu katmanda toplanır.


### 5.6. process() – Dışarı Açılan Tek Kapı

`process()` fonksiyonu, dış dünyadan görünen ana API’dir.

İç akış:

1. `preprocess(frame.image)` → input tensor
2. `inference(input_tensor)` → ham model çıktısı
3. `postprocess(raw_output)` → `std::vector<DetectedObject>`

Dışarıdan kullanım örneği kafada şöyle canlanmalıdır:

```cpp
PerceptionNode perception("model.pt", config);

while (true) {
    ImageFrame frame = grab_frame_from_camera();
    auto detections = perception.process(frame);
    // detections → logla, çiz, publish et, vs.
}
```

Bu tasarımın avantajları:

- Algılama mantığı tek bir sınıf içinde toplanır.
- Aynı `PerceptionNode` hem:
  - Komut satırı programında,
  - ROS2 node içinde,
  - Multi-thread pipeline’da,
  - Unit test senaryolarında
  kullanılabilir.


## 6. CNN ve YOLO Bu İskelete Nasıl Oturuyor?

### 6.1. CNN Classifier Senaryosu

- Girdi: Tek nesne içeren crop veya tüm resim.
- Çıktı: Sınıf olasılıkları.

Pipeline akışı değişmez:

1. `preprocess()` → 224x224 resize, normalize, vs.
2. `inference()` → `[1, num_classes]` tensor.
3. `postprocess()` → tek bir `DetectedObject` (veya hiç).

Burada `DetectedObject` sayısı genelde 1’dir.
`bbox` isteğe bağlı olarak tüm resmi kapsayacak şekilde kullanılabilir.

### 6.2. YOLO / Nesne Tespit Senaryosu

- Girdi: Kamera görüntüsü.
- Çıktı: Birden fazla nesne için bbox + sınıf + skor.

Yine akış aynı:

1. `preprocess()` → 640x640 letterbox, normalize.
2. `inference()` → ham YOLO çıktısı.
3. `postprocess()` → threshold + NMS + `DetectedObject` listesi.

Burada her frame için `DetectedObject` sayısı değişebilir (0, 5, 20, ...).

Önemli nokta:

> **İskelet (constructor, preprocess, inference, postprocess, process) aynı kalır.**
>
> Sadece `postprocess()` içeriği, modelin çıktı formatına göre uyarlanır.


## 7. Gerçek Zamanlı Sistemler Açısından Neden Doğru Tasarım?

Bu modüler yapı, gerçek zamanlı sistemlerde şu avantajları sağlar:

### 7.1. Sorumluluk Ayrımı (Separation of Concerns)

- Kamera okuma / IO → ayrı modül veya thread
- Algılama (`PerceptionNode`) → ayrı modül
- Çizim, UI, loglama → ayrı modül

Bu sayede kod:

- Daha okunabilir,
- Daha kolay test edilebilir,
- Hata ayıklaması daha rahat yapılabilir hale gelir.

### 7.2. Modülerlik ve Değişim Kolaylığı

- Aynı API korunarak:
  - CPU’dan CUDA’ya geçilebilir.
  - CNN’den YOLO’ya geçilebilir.
  - Model path’i, input boyutu, threshold’lar değiştirilebilir.

Tüm bu değişiklikler sadece `PerceptionNode` içinde yapılır.
Dışarıdaki `perception.process(frame)` çağrılarını değiştirmek gerekmez.

### 7.3. Multi-thread ve ROS2 Uygunluğu

- Bir thread sadece kamera frame’lerini toplar, kuyruğa koyar.
- Bir thread bu kuyruktan `ImageFrame` alır ve `process()` çağırır.
- ROS2 subscriber callback’i içinde yine sadece `process()` çağrılır,
  sonra sonuçlar başka bir topic’e publish edilir.

Burada `PerceptionNode`, başlı başına **bağımsız, yeniden kullanılabilir** bir bileşendir.

### 7.4. Performans ve Deterministik Davranış

- Model sadece bir kere yüklenir, tekrar tekrar kullanılabilir.
- `process()` içinde her seferinde aynı adımlar çalışır,
  bu da latency ölçümü ve optimizasyonu kolaylaştırır.
- Heap allocation’lar azaltılarak daha kararlı FPS elde edilebilir.

### 7.5. Test Edilebilirlik

- Unit test: Belirli bir `ImageFrame` için beklenen `DetectedObject` sayısı ve tipini kontrol etmek.
- Offline test: Video veya kayıtlı frame dizileri üzerinde `PerceptionNode`’u çalıştırıp logları incelemek.

Bu sayede hem araştırma/deneme aşamasında hem de üretim ortamında aynı iskelet kullanılabilir.


## 8. Özet ve Sonraki Aşama

Bu teorik iskelet ile şunları netleştirdik:

- **ImageFrame** → Görüntü + zaman bilgisi + kamera bağlamı.
- **DetectedObject** → Model çıktısının anlamlı temsili (sınıf, bbox, skor).
- **PerceptionNode** →
  - Modeli yükleyen ve yöneten sınıf,
  - `preprocess`, `inference`, `postprocess` adımlarını içinde barındıran modül,
  - Dışarıya **sadece** `process(frame)` fonksiyonunu açan temiz bir arayüz.

Aynı yapıyı:

- CNN classifier,
- YOLO tabanlı nesne tespiti,
- İleride farklı ağ tipleri

için kullanmak mümkündür.

---

# 1) Perception Node Mimarisi (Blok Diagram – Görsel Tasarım)

Aşağıdaki tasarım, gerçek bir otonom sistemin perception pipeline’ını temsil ediyor:

```bash
┌─────────────────────────┐
│       Camera Sensor      │
│   (cv::Mat + metadata)   │
└──────────────┬───────────┘
               │  ImageFrame
               ▼
       ┌───────────────────┐
       │   PerceptionNode  │
       │  (LibTorch + CV)  │
       └───────┬───────────┘
               │
               ▼
   ┌─────────────────────────────┐
   │     preprocess(frame)       │
   │  cv::Mat → torch::Tensor    │
   └───────────────┬─────────────┘
                   │
                   ▼
   ┌─────────────────────────────┐
   │       inference(tensor)     │
   │     model.forward()         │
   │   TorchScript computation   │
   └───────────────┬─────────────┘
                   │
                   ▼
   ┌─────────────────────────────┐
   │     postprocess(output)     │
   │  bbox + class + score       │
   │  NMS / filtering            │
   └───────────────┬─────────────┘
                   │
                   ▼
     ┌─────────────────────────┐
     │  DetectedObject List    │
     │  (class, bbox, score)   │
     └───────────────┬─────────┘
                     │
                     ▼
      ┌────────────────────────────┐
      │   Downstream Modules       │
      │  (Tracking, Planning, UI)  │
      └────────────────────────────┘
```

- Bu diyagram, perception node’un gerçek zamanlı bir sistemde nerede durduğunu, nasıl veri aldığını ve nasıl veri ürettiğini açık biçimde gösterir. 

----
# 2) PerceptionNode.process() Akış Şeması

Bu akış şeması direkt olarak process(ImageFrame) fonksiyonunun içsel işleyişini gösterir:



```cpp
                   ┌───────────────────────┐
                   │  Start process(frame) │
                   └───────────────┬───────┘
                                   │
                                   ▼
                     ┌──────────────────────┐
                     │  Preprocess Stage    │
                     │  - resize            │
                     │  - normalize         │
                     │  - NHWC → NCHW       │
                     └──────────────┬───────┘
                                    │  tensor
                                    ▼
                     ┌──────────────────────┐
                     │   Inference Stage    │
                     │  model.forward()     │
                     ○  NoGradGuard         │
                     └──────────────┬───────┘
                                    │ raw output
                                    ▼
                     ┌────────────────────────┐
                     │    Postprocess Stage    │
                     │  - decode bbox          │
                     │  - score threshold      │
                     │  - NMS                  │
                     │  - DetectedObject list  │
                     └───────────────┬────────┘
                                     │
                                     ▼
                          ┌────────────────────┐
                          │ return detections  │
                          └────────────────────┘


* Bu akış sayesinde tek bir process() çağrısının içindeki tüm alt aşamaları görsel olarak anlayabilirsin.

---

# 3) Sistem Seviyesinde Akış (Kamera → Algılama → Uygulama)


* Bunu özellikle ROS2, multi-thread veya otomotiv algılama pipeline’ları için kullanırsın.

```bash
Camera Thread                  Perception Thread               Application Thread
───────────────               ────────────────────             ────────────────────
Grab frame                     perception.process(frame)        Track / Plan / UI
       │                                  │                         │
       ▼                                  ▼                         ▼
┌────────────┐                   ┌─────────────────┐         ┌───────────────────┐
│ ImageFrame │─────────────────▶ │ DetectedObjects │ ───────▶│ Downstream logic │
└────────────┘                   └─────────────────┘         └───────────────────┘
```
* Bu diyagram gerçek zamanlı sistemlerde thread separation mantığını da doğrudan gösterir.

----

---
## Proje Yapısı 
---

---

# Bu Perception projesinin yapısını böyle kuruyorum
### 1. Önce hedefimi net koyuyorum

* “Benim amacım, LibTorch + OpenCV kullanan algılama kodumu temiz bir modüle dönüştürmek.
* Projede, algılama tarafını PerceptionNode diye ayrı bir sınıfın içine alacağım.
* Dışarıdan sadece perception.process(frame) diye çağırmak isteyeceğim.”

Bu hedefe göre proje yapısını sade tutuyorum.İlerleyen projelerde daha karmaşık yapılara geçebiliriz.

### 2. Klasör ve dosya yapısını oluşturuyorum

Projenin ana klasörünü şöyle düşünüyorum:
```bash
PerceptionProject/
  CMakeLists.txt
  src/
    main.cpp
    perception_types.hpp
    perception_node.hpp
    perception_node.cpp
```


* CMakeLists.txt → Projenin derleme ayarlarını burada yapıyorum (LibTorch + OpenCV linkleri).

* src/main.cpp → Sadece “uygulama” tarafı; kamera açma, frame alma, PerceptionNode kullanımı burada.

src/perception_types.hpp → Veri tiplerimi burada tanımlıyorum:

* ImageFrame

* DetectedObject

* PerceptionConfig

**src/perception_node.hpp / .cpp → Asıl algılama modülüm:**

* Constructor → modeli yüklüyor.

* process() → tüm pipeline’ı (preprocess, inference, postprocess) çalıştırıyor.

Bunun mantığı:
* Algılama mantığı tek bir sınıfta, veri tipleri ayrı bir header’da, uygulama ise main.cpp’de duruyor.

